In [1]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import roc_curve,roc_auc_score,auc
import math,scipy
import tensorly as tl
from tensorly.base import partial_unfold,partial_tensor_to_vec,tensor_to_vec,vec_to_tensor
from tensorly.tenalg import khatri_rao,kronecker
from tensorly.tucker_tensor import tucker_to_tensor
from scipy.linalg import fractional_matrix_power

In [2]:
def b(x,k,knotes,order):
    if k+1 <= order:
        return x**(k)
    else:
        x_ = x.copy()
        x_ = x_.reshape(-1)
        for i in range(len(x_)):
            if x_[i] > knotes[k-order]:
                x_[i] = (x_[i]-knotes[k-order])**(order-1)
            else:
                x_[i] = 0
        return x_.reshape(31,31,36)
def Phi(X,knotes_length,order):
    (N,I,J,M) = X.shape
    Phi_X = np.zeros((N,I,J,M,knotes_length+order-1))
    for i in range(N):
        knotes = np.linspace(np.min(X[i]),np.max(X[i]),(knotes_length+2))[1:(knotes_length+1)]
        for k in range(1,knotes_length+order):
            Phi_X[i,:,:,:,k-1] = b(X[i],k,knotes,order)    
    return Phi_X

In [3]:
class LS_SVM:
    def __init__(self,C):
        self.C = C
    def kernelTrans(self,X):
        return np.dot(X,X.T)
    def fit(self,X,y):
        X = np.array(X)
        n,m = X.shape  
        Diag_y = np.diag(list(y))
        y = np.array(y).reshape(n,1)
        unit = np.array(np.ones((n,1)))  #[1,1,...,1].T
        I = np.eye(n)
        alpha = np.array(np.zeros((n,1)))
        zero = np.array(np.zeros((1,1)))
        X_y = np.dot(Diag_y,X)
        H_matrix = self.kernelTrans(X_y)
        upmat = np.hstack((zero,(-y).T))
        downmat = np.hstack((y,H_matrix + I/self.C))
        completemat = np.vstack((upmat,downmat))  #lssvm中求解方程的左边矩阵
        rightmat = np.vstack((zero,unit)) # lssvm中求解方程的右边矩阵
        b_alpha = np.dot(np.linalg.inv(completemat),rightmat)
        sum_list = []
        for i in range(n):
            sum_list.append(b_alpha[i+1,0]*y[i]*X[i,:])
        W_=np.array(sum_list)
        self.W = np.sum(W_,axis=0)
        self.b = b_alpha[0,0]
        return None
    def predict(self,X_test):
        y_predict = np.dot(X_test,self.W) + self.b
        return np.sign(y_predict)
    def score(self,X_test,y_test):
        return accuracy_score(y_test,self.predict(X_test))

In [4]:
class R_STM:
    def __init__(self, weight_rank=6, C=0.001):
        self.weight_rank = weight_rank
#         self.tol = tol
        self.C = C
    def fit(self,X,y):
        W = []
        C_range = np.linspace(0.01,0.05,5)
        dict_param = dict(C = C_range)
        for i in range(1, tl.ndim(X)):  
            W.append(tl.tensor(np.zeros((X.shape[i], self.weight_rank))))
        for w in W:
            w = self.matrix_(w)
        # W的范数列表
        norm_W = []
        weights = tl.ones(self.weight_rank)
        iteration = 0
        while True:
            for k in range(len(W)):
                U_out_k = khatri_rao(W, skip_matrix=k, reverse=True)
                A_k = np.dot(U_out_k.T,U_out_k)
                A_k_1 = fractional_matrix_power(A_k, 0.5)
                A_k_2 = self.A_(A_k)
                U_k_ = np.dot(W[k],A_k_1)
                U_A = np.dot(U_out_k,A_k_2)
                
                X_copy = []
                for j in range(len(X)):
                    X_k_ = tl.unfold(X[j],mode=k)
                    X_copy.append(np.dot(X_k_,U_A))
                X_k = np.array(X_copy)
                X_vec = partial_tensor_to_vec(X_k,skip_begin=1)
                
                update = SVC(C=self.C,kernel="linear")#训练模型
                update.fit(X_vec,y)
                U_k_ = vec_to_tensor(update.coef_.reshape(-1,1),U_k_.shape)
                W[k] = np.dot(U_k_,A_k_2)
                
            b = update.intercept_
            weight_tensor_ = tl.kruskal_to_tensor((weights, W))
            norm_W.append(tl.norm(weight_tensor_, 2))
            iteration += 1
            if iteration > 1:
                weight_evolution = abs(norm_W[-1] - norm_W[-2]) / norm_W[-1]
#                 print("第{}次迭代结束，权值误差为{}".format(iteration,weight_evolution))
                if (weight_evolution <= 10e-4
                   ):
                    if 1:
                        print('CP_Model is ok')
                        print('\nConverged in {} iterations'.format(iteration))
                    break  
        self.W = W
        self.b = b
        self.weight = tl.kruskal_to_tensor((weights,W))
        return self
    def predict(self,X_test):
        y_predict = tl.dot(partial_tensor_to_vec(X_test,skip_begin=1),tl.tensor_to_vec(self.weight)) + self.b
        return np.sign(y_predict)
    def score(self,X_test,y_test):
        return accuracy_score(y_test,self.predict(X_test))
    def matrix_(self,W):
        for i in range(W.shape[0]):
            for j in range(W.shape[1]):
                if i == j:
                    W[i,j]=1
        return W
    def A_(self,matrix):
        U,S,V = np.linalg.svd(matrix)
        Diag = np.diag(S**(-0.5))
        return np.dot(np.dot(U,Diag),V)
    def get_params(self, deep=False):

        params = {'C': self.C,
                  'weight_rank': self.weight_rank,
                  }
        return params

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self